# Course project


**Основное**
- Дедлайн - 19 февраля 23:59
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. В решении должны быть отчетливо видна метрика на новом тестовом сете из файла retail_test1.csv, то есть вам нужно для всех юзеров из этого файла выдать выши рекомендации, и посчитать на actual покупках precision@5. 

**!! Мы не рассматриваем холодный старт для пользователя, все наши пользователя одинаковы во всех сетах, поэтому нужно позаботиться об их исключении из теста.**


**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)  
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель - следите за валидацией 

# Import libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

## Read data

In [2]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

In [3]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


# Set global const

In [4]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

# N = Neighbors
N_PREDICT = 110 

# Process features dataset

In [5]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

# Split dataset for train, eval, test

In [6]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 7 недель -- | -- 4 недель -- 
# подобрать размер 2-ого датасета (7 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
VAL_MATCHER_WEEKS = 11
VAL_RANKER_WEEKS = 3

In [7]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]

# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [8]:
data_train_matcher.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [9]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [10]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [11]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (1968369, 12) Users: 2498 Items: 80902
val_matcher
Shape: (310121, 12) Users: 2293 Items: 35272
train_ranker
Shape: (310121, 12) Users: 2293 Items: 35272
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [12]:
# выше видим разброс по пользователям и товарам и дальше мы перейдем к warm-start (только известные пользователи)

In [13]:
data_val_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
1963047,2375,40186322971,559,827667,2,6.00,364,-1.78,34,81,0.0,0.0
1963048,2375,40186322971,559,834631,1,1.69,364,0.00,34,81,0.0,0.0


# Prefilter items

In [14]:
n_items_before = data_train_matcher['item_id'].nunique()

n_popular = 1500

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=n_popular)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/Users/ivanbovsunovskii/Downloads/course_project (1)/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 80902 to 1501


# Make cold-start to warm-start

In [15]:
# ищем общих пользователей
common_users = list(set(data_train_matcher.user_id.values)&(set(data_val_matcher.user_id.values))&set(data_val_ranker.user_id.values))

#выбираем товары из подготовленного набора
prefiltered_items = list(set(data_train_matcher['item_id']))

# оставляем общих пользователей
data_train_matcher = data_train_matcher[data_train_matcher.user_id.isin(common_users)]
data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

#оставляем подготовленные товары по всем массивам данных
data_val_matcher = data_val_matcher[data_val_matcher.item_id.isin(prefiltered_items)]
data_train_ranker = data_train_ranker[data_train_ranker.item_id.isin(prefiltered_items)]
data_val_ranker = data_val_ranker[data_val_ranker.item_id.isin(prefiltered_items)]


print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (739351, 13) Users: 1981 Items: 1501
val_matcher
Shape: (64997, 12) Users: 1902 Items: 1415
train_ranker
Shape: (64997, 12) Users: 1902 Items: 1415
val_ranker
Shape: (26084, 12) Users: 1801 Items: 1369


In [16]:
data_train_matcher.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0,2.99
11,1364,26984896261,1,999999,1,2.19,31742,0.00,1520,1,0.0,0.0,2.19
12,1364,26984896261,1,999999,1,2.99,31742,-0.40,1520,1,0.0,0.0,2.99
13,1364,26984896261,1,999999,1,3.09,31742,0.00,1520,1,0.0,0.0,3.09
14,1364,26984896261,1,999999,1,2.50,31742,-0.99,1520,1,0.0,0.0,2.50


# Init/train recommender

In [17]:
recommender = MainRecommender(data_train_matcher, weighting='bm25', B=0.895, 
                              n_factors=200, regularization=0.001, iterations=35, num_threads=4, 
                              K=1)

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/1501 [00:00<?, ?it/s]

### Варианты, как получить кандидатов

Можно потом все эти варианты соединить в один

(!) Если модель рекомендует < N товаров, то рекомендации дополняются топ-популярными товарами до N

# Eval recall of matching

### Измеряем recall@k

In [18]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[856942, 857006, 928342, 1041796, 1082185, 109..."
1,3,"[910032, 913210, 965766, 946839]"


In [19]:
def recommended_items(calc_type, users_list, number_items, function):
    if calc_type == 0:
        return [function(user, number_items) for user in users_list]
    elif calc_type == 1:
        return users_list.apply(lambda x: function(x, N=number_items))
    else:
        return list(map(lambda x: function(x, N=number_items), users_list))

In [20]:
#result_eval_matcher['own_rec'] = recommended_items(0, result_eval_matcher[USER_COL], N_PREDICT, recommender.get_own_recommendations)

### Пример оборачивания

In [21]:
def evalRecall(df_result, target_col_name, recommend_model, result_col_name='result', 
               actual_col = 'actual', N_PREDICT=50, inplace = False):
    df_result[result_col_name] = recommended_items(0, df_result[target_col_name], N_PREDICT, recommend_model)
    result = np.mean([recall_at_k(a, b, k=N_PREDICT) for a,b in zip(df_result[result_col_name], df_result[actual_col])])
    if not(inplace):
        df_result.drop(columns=result_col_name, inplace=True)
    return result

In [22]:
def evalPrecision(df_result, target_col_name, recommend_model, result_col_name='result', 
                  actual_col = 'actual', N_PREDICT=50, inplace = False):
    df_result[result_col_name] = recommended_items(0, df_result[target_col_name], N_PREDICT, recommend_model)
    result = np.mean([precision_at_k(a, b, k=N_PREDICT) for a,b in zip(df_result[result_col_name], df_result[actual_col])])
    if not(inplace):
        df_result.drop(columns=result_col_name, inplace=True)
    return result

In [23]:
# evalRecall(result_eval_matcher, USER_COL, recommender.get_own_recommendations)

In [24]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [25]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

### Recall@50 of matching

In [26]:
TOPK_RECALL = 50

In [27]:
#sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)

### Precision@5 of matching

In [28]:
TOPK_PRECISION = 5

In [29]:
#sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

# Ranking part

### Обучаем модель 2-ого уровня на выбранных кандидатах

- Обучаем на data_train_ranking
- Обучаем *только* на выбранных кандидатах

In [30]:
# -- давние покупки -- | -- 7 недель -- | -- 4 недель -- 

## Подготовка данных для трейна

In [31]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [32]:
# собираем кандитатов с первого этапа (matcher)
#df_match_candidates['candidates'] = recommended_items(0, df_match_candidates[USER_COL], N_PREDICT, recommender.get_own_recommendations)
df_match_candidates['candidates'] = recommended_items(0, df_match_candidates[USER_COL], N_PREDICT, recommender.get_als_recommendations)


In [33]:
df_match_candidates.head(2)

,user_id,candidates
0,2375,"[1046545, 999104, 1000753, 899624, 902172, 101..."
1,989,"[8090509, 985999, 957951, 1020581, 893018, 880..."


In [34]:
# разворачиваем товары
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [35]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [36]:
df_match_candidates.head(4)

,user_id,item_id
0,2375,1046545
0,2375,999104
0,2375,1000753
0,2375,899624


### Check warm start

In [37]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (209220, 2) Users: 1902 Items: 1482


### Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [38]:
data_train_ranker.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
1963047,2375,40186322971,559,827667,2,6.00,364,-1.78,34,81,0.0,0.0
1963052,2375,40186322971,559,896613,1,4.03,364,-1.02,34,81,0.0,0.0
1963053,2375,40186322971,559,899624,4,15.96,364,0.00,34,81,0.0,0.0
1963063,2375,40186322971,559,6034857,1,3.99,364,0.00,34,81,0.0,0.0
1966158,989,40186669139,559,823990,1,3.50,356,-2.64,26,81,0.0,0.0


In [39]:
#df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL, 'basket_id', 'quantity', 'sales_value', 'retail_disc', 'week_no', 'store_id']].copy()
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL, 'basket_id', 'quantity', 'sales_value', 'retail_disc', 'week_no', 'store_id']].copy()
df_ranker_train['target'] = 1  # тут только покупки 

df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

df_ranker_train['target'].fillna(0, inplace= True)

In [40]:
df_ranker_train.target.value_counts()

0.0    196942
1.0     22234
Name: target, dtype: int64

(!) На каждого юзера 50 item_id-кандидатов

In [41]:
df_ranker_train['target'].mean()

0.10144358871409279

## Подготавливаем фичи для обучения модели

### Описательные фичи

In [42]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [43]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [44]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,basket_id,quantity,sales_value,retail_disc,week_no,store_id,target,manufacturer,...,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2375,1046545,NaN,NaN,NaN,NaN,NaN,NaN,0.0,69,...,POTATOES,POTATOES RUSSET (BULK&BAG),10 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2375,999104,4.028339e+10,1.0,2.69,-0.15,82.0,364.0,1.0,1194,...,REFRGRATD JUICES/DRNKS,DAIRY CASE 100% PURE JUICE - O,64OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Фичи user_id:**
    - Средний чек
    - Средняя сумма покупки 1 товара в каждой категории
    - Кол-во покупок в каждой категории
    - Частотность покупок раз/месяц
    - Долю покупок в выходные
    - Долю покупок утром/днем/вечером

**Фичи item_id**:
    - Кол-во покупок в неделю
    - Среднее ол-во покупок 1 товара в категории в неделю
    - (Кол-во покупок в неделю) / (Среднее ол-во покупок 1 товара в категории в неделю)
    - Цена (Можно посчитать из retil_train.csv)
    - Цена / Средняя цена товара в категории
    
**Фичи пары user_id - item_id**
    - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
    - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
    - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

### Поведенческие фичи

##### Чтобы считать поведенческие фичи, нужно учесть все данные что были до data_val_ranker

In [45]:
df_join_train_matcher.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [46]:
df_ranker_train.head()

,user_id,item_id,basket_id,quantity,sales_value,retail_disc,week_no,store_id,target,manufacturer,...,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2375,1046545,NaN,NaN,NaN,NaN,NaN,NaN,0.0,69,...,POTATOES,POTATOES RUSSET (BULK&BAG),10 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2375,999104,4.028339e+10,1.0,2.69,-0.15,82.0,364.0,1.0,1194,...,REFRGRATD JUICES/DRNKS,DAIRY CASE 100% PURE JUICE - O,64OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2375,1000753,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2822,...,BEEF,SELECT BEEF,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2375,899624,4.018632e+10,4.0,15.96,0.00,81.0,364.0,1.0,69,...,POTATOES,POTATOES RUSSET (BULK&BAG),10 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2375,899624,4.059502e+10,2.0,5.98,-2.00,85.0,364.0,1.0,69,...,POTATOES,POTATOES RUSSET (BULK&BAG),10 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## !!! Пока выполните нотбук без этих строк, потом вернитесь и запустите их, обучите ранкер и посмотрите на метрики с ранжированием

In [47]:
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)

#df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)

#df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_user')/df_join_train_matcher.user_id.nunique(), how='left',on=ITEM_COL)

#df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)

#df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

#df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_baskter')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)

#df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

#df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


In [48]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [49]:
X_train['quantity'].fillna(value=0, inplace=True)
X_train['sales_value'].fillna(value=0, inplace=True)
X_train['retail_disc'].fillna(value=0, inplace=True)
X_train['age_desc'].fillna(value='19-65+', inplace=True)
X_train['marital_status_code'].fillna( value='C', inplace=True)
X_train['income_desc'].fillna( value='0-250K+', inplace=True)
X_train['homeowner_desc'].fillna( value='Unknown', inplace=True)
X_train['hh_comp_desc'].fillna( value='Unknown', inplace=True)
X_train['household_size_desc'].fillna( value='Unknown', inplace=True)
X_train['kid_category_desc'].fillna( value='None/Unknown', inplace=True)
X_train['store_id'] = X_train['store_id'].astype(str)
X_train['basket_id'] = X_train['basket_id'].astype(str)
X_train.replace('nan', 'U', inplace=True)


In [50]:
X_train['week_no'].fillna(value=0, inplace=True)
X_train['week_no'] = X_train['week_no'].astype(int)

In [51]:
#unimportant_features = ['brand', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product', 
#                        'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc', 'hh_comp_desc', 
#                        'household_size_desc', 'kid_category_desc', 'item_quantity_per_week'
#                       ]
unimportant_features = ['basket_id']
X_train.drop(columns = unimportant_features, inplace=True)

In [52]:
num_feats = [col for col in X_train.columns if (X_train[col].dtype == 'int64') or (X_train[col].dtype == 'float64')]
num_feats = set(num_feats) - set(['manufacturer','user_id', 'item_id', 'week_no'])
num_feats

{'item_freq',
 'item_quantity_per_user',
 'item_quantity_per_week',
 'quantity',
 'retail_disc',
 'sales_value',
 'total_item_sales_value',
 'total_quantity_value'}

In [53]:
cat_feats = X_train.columns[0:].tolist()
cat_feats = list(set(cat_feats) - set(num_feats))
cat_feats

['curr_size_of_product',
 'week_no',
 'sub_commodity_desc',
 'household_size_desc',
 'department',
 'hh_comp_desc',
 'brand',
 'commodity_desc',
 'marital_status_code',
 'user_id',
 'age_desc',
 'item_id',
 'income_desc',
 'manufacturer',
 'homeowner_desc',
 'store_id',
 'kid_category_desc']

In [54]:
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [55]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219176 entries, 0 to 219175
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   user_id                 219176 non-null  category
 1   item_id                 219176 non-null  category
 2   quantity                219176 non-null  float64 
 3   sales_value             219176 non-null  float64 
 4   retail_disc             219176 non-null  float64 
 5   week_no                 219176 non-null  category
 6   store_id                219176 non-null  category
 7   manufacturer            219176 non-null  category
 8   department              219176 non-null  category
 9   brand                   219176 non-null  category
 10  commodity_desc          219176 non-null  category
 11  sub_commodity_desc      219176 non-null  category
 12  curr_size_of_product    219176 non-null  category
 13  age_desc                219176 non-null  category
 14  mari

## Обучение модели ранжирования

In [56]:
# week for validation
#data = X_train
#data['target'] = y_train

#X_val = data[data['week_no'] >= data['week_no'].max() - 2]
#X_train = data[data['week_no'] < data['week_no'].max() - 2]

#y_train = X_train[['target']]
#X_train = X_train.drop('target', axis=1)

#y_val = X_val[['target']]
#X_val = X_val.drop('target', axis=1)

In [57]:
#feature_weight = {"week_no":0.005,"store_id":0.01, "quantity":0.01, "sales_value":0.01, 
#                  "retail_disc":0.01, "basket_id":0.11, "retail_disc":0.005, "item_id":0.1, "user_id":0.1}
feature_weight = {"week_no":0.08, "sales_value":0.083, "quantity":0.084, "store_id":0.08,
                  "retail_disc":0.5}

#feature_weight = {"quantity":1}

model = CatBoostClassifier(
    iterations=70,
    learning_rate=0.05,
    depth=10,
    random_seed=12,
    logging_level='Silent',
    cat_features=cat_feats,
    feature_weights = feature_weight,
    custom_metric=['Logloss', 'Precision', 'F1', 'Recall']
)
#use_best_model=True

model.fit(
    X_train, y_train,
    plot=True
)
#eval_set=(X_val, y_val),

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [58]:
model.get_feature_importance(prettified = True)

,Feature Id,Importances
0,retail_disc,25.520207
1,quantity,23.589461
2,week_no,17.987041
3,sales_value,6.710965
4,user_id,4.113506
5,item_quantity_per_week,3.151100
6,commodity_desc,2.910157
7,item_quantity_per_user,2.846037
8,department,2.699033
9,item_freq,2.447351


In [59]:
#train_preds = model.predict_proba(data.drop('target', axis=1))
train_preds = model.predict_proba(X_train)

In [60]:
df_ranker_predict = df_ranker_train.copy()

In [61]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

## Подведем итоги

    Мы обучили модель ранжирования на покупках из сета data_train_ranker и на кандитатах от als_recommendations, что является тренировочным сетом, и теперь наша задача предсказать и оценить именно на тестовом сете.

# Evaluation on test dataset

In [63]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[856942, 865456, 971585, 979707, 990656, 10049..."
1,6,"[870315, 956672, 948650, 1024306, 1037863, 107..."


## Eval matching on test dataset

In [64]:
%%time
result_eval_ranker['own_rec'] = recommended_items(0, result_eval_ranker[USER_COL], N_PREDICT, recommender.get_own_recommendations)
result_eval_ranker['als_rec'] = recommended_items(0, result_eval_ranker[USER_COL], N_PREDICT, recommender.get_als_recommendations)

CPU times: user 54.7 s, sys: 1.37 s, total: 56.1 s
Wall time: 35.1 s


In [65]:
f'TOPK_PRECISION = {TOPK_PRECISION}'

'TOPK_PRECISION = 5'

In [66]:
# померяем precision только модели матчинга, чтобы понимать влияение ранжирования на метрики

#sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

## Eval re-ranked matched result on test dataset
    Вспомним df_match_candidates сет, который был получен own_recommendations на юзерах, набор пользователей мы фиксировали и он одинаков, значи и прогноз одинаков, поэтому мы можем использовать этот датафрейм для переранжирования.
    

In [67]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [68]:
result_eval_ranker['reranked_als_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

In [69]:
print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True),  f'TOPK_PRECISION = {TOPK_PRECISION}', sep='\n')

('reranked_als_rec', 0.19039999999999746)
('als_rec', 0.14392004441976602)
('own_rec', 0.03076068850638542)
TOPK_PRECISION = 5


/Users/ivanbovsunovskii/Downloads/course_project (1)/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


In [70]:
#precision@5 >= 0.25

In [71]:
# смотрим на метрики выше и сравниваем что с ранжированием и без, добавляем фичи и то же смотрим

# Оценка на тесте для выполнения курсового проекта

In [72]:
df_test = pd.read_csv('retail_test1.csv')
#df_transactions = pd.read_csv('retail_train.csv')

In [73]:
df_test = df_test[df_test.user_id.isin(common_users)]

In [74]:
№.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0


In [84]:
result_test = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]
result_test.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,3,"[827683, 908531, 989069, 1071377, 1080155, 109..."


In [76]:
%%time
#result_test['own_rec'] = recommended_items(0, result_test[USER_COL], N_PREDICT, recommender.get_own_recommendations)
#result_test['als_rec'] = recommended_items(0, result_test[USER_COL], N_PREDICT, recommender.get_als_recommendations)

CPU times: user 51.8 s, sys: 1.67 s, total: 53.5 s
Wall time: 46 s


In [77]:
#sorted(calc_precision(result_test, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

In [85]:
result_test['reranked_own_rec'] = result_test[USER_COL].apply(lambda user_id: rerank(user_id))

In [86]:
q = sorted(calc_precision(result_test, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

/Users/ivanbovsunovskii/Downloads/course_project (1)/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


In [87]:
print(*sorted(calc_precision(result_test, TOPK_PRECISION), key=lambda x: x[1], reverse=True),  f'TOPK_PRECISION = {TOPK_PRECISION}', sep='\n')

('reranked_own_rec', 0.14062877871825788)
TOPK_PRECISION = 5
